<a href="https://colab.research.google.com/github/qahaidari/Keras-classification-with-finetuned-vgg16-flask-deployment/blob/main/classification_vgg16_finetuned_frontend_flask_singleImage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install flask-ngrok

In the below code, the URL of the currently running backend service is inserted for the HTTP POST method. 

In [4]:
# a simple frontend with flask for sending a text to backend for processing
from flask_ngrok import run_with_ngrok
from flask import Flask

app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run
@app.route("/")
def home():

    return '''<head>
                <title>A simple web application with Flask</title>
                <style>
                  * {
                      font-size:15px;
                  }
                </style>
              </head>
              
              <body>
                <input id="name-input" type="text"/>
                <button id="name-button">Submit Name</button>
                <p id="greeting"></p>
                <script src="https://code.jquery.com/jquery-3.3.1.min.js"></script>
                <script>
                  $("#name-button").click(function(event){
                    let message = {
                      name: $("#name-input").val()
                    }
                    $.post("http://bc746e24d65a.ngrok.io", JSON.stringify(message), function(response){
                      $("#greeting").text(response.greeting);
                      console.log(response); 
                    });
                  });
                </script>
              </body>'''

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://ac4a2088e54c.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


In [5]:
# A web application running on Flask to send one image of cats or dogs to a finetuned VGG16 model running on Flask
from flask_ngrok import run_with_ngrok
from flask import Flask

app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run
@app.route("/")
def home():

    return '''<head>
                <title>Transfer learning with VGG16 for one image</title>
                <style>
                  * {
                      font-size:15px;
                  }
                </style>
              </head>
              
              <body>
                <input id="image-selector" type="file">    
                <button id="predict-button">Predict</button>
                <p style="font-weight:bold">Predictions</p>    
                <p>Dog: <span id="dog-prediction"></span></p>
                <p>Cat: <span id="cat-prediction"></span></p>
                <img id="selected-image" src=""/>

                <script src="https://code.jquery.com/jquery-3.3.1.min.js"></script>
                <script>
                  let base64Image;
                  $("#image-selector").change(function() {
                    let reader = new FileReader();
                    reader.onload = function(e) {
                      let dataURL = reader.result;
                      $('#selected-image').attr("src", dataURL);
                      base64Image = dataURL.replace("data:image/jpeg;base64,","");
                      console.log(base64Image);
                    }
                    reader.readAsDataURL($("#image-selector")[0].files[0]);
                    $("#dog-prediction").text("");
                    $("#cat-prediction").text("");
                  });

                  $("#predict-button").click(function(){
                    let message = {
                      image: base64Image
                    }
                    console.log(message);
                    $.post("http://ba8cf2cc7cdf.ngrok.io", JSON.stringify(message), function(response){
                      $("#dog-prediction").text(response.prediction.dog.toFixed(6));
                      $("#cat-prediction").text(response.prediction.cat.toFixed(6));
                      console.log(response);
                    });
                  });       
                </script>
              </body>'''

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://98109bc865b2.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [13/Apr/2021 22:21:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2021 22:21:30] "GET /favicon.ico HTTP/1.1" 404 -


**How to fix the CORS error**

By checking the console on your browser, you can see the status and result of the HTTP requests that you are sending to the web service. If the following CORS error is returned:

Access to XMLHttpRequest at 'http://ba8cf2cc7cdf.ngrok.io/' from origin 'http://98109bc865b2.ngrok.io' has been blocked by CORS policy: No 'Access-Control-Allow-Origin' header is present on the requested resource.

, it can be solved by adding an extension to the browser. Below tutorial shows how to fix this error.

[3 Ways to Fix the CORS Error — and How 
the Access-Control-Allow-Origin Header 
Works](https://medium.com/@dtkatz/3-ways-to-fix-the-cors-error-and-how-access-control-allow-origin-works-d97d55946d9#:~:text=The%20error%20stems%20from%20a,the%20browser's%20cookie%20storage%20system.)

The way I solved this problem was by adding moesif CORS extension to 
my Chrome browser and setting its status to ON. After refreshing my backend and frontend services, I could receive the prediction for my input image.